In [1]:
%load_ext autoreload
%autoreload 2

In [156]:
import json
from mlflow.models import infer_signature
from label_legends.preprocess import holdout, load_test, tfidf_pipeline, token_lists, transform, load_test
from xgboost import XGBClassifier
import logging
import numpy as np
import mlflow
from mlflow import MlflowClient
from polars import DataFrame

import polars
from label_legends.result import calculate_scores, download_predictions, get_experiment, get_latest_run, load_predictions
from label_legends.util import RESOURCE, ROOT

np.set_printoptions(threshold=1000)
logging.basicConfig(level=logging.INFO)

In [3]:
val, tra = holdout()
tra = transform(tra)
val = transform(val)

test = transform(load_test().collect())

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
tra

id,text,tokens,token_ids,label
i64,str,list[str],list[i64],i64
26016,"""#VoteDemOut seditionist wanna …","[""#"", ""votedemout"", … ""ven""]","[7, 3000, … 3000]",0
41060,"""Cerno hops from wave to wave: …","[""cerno"", ""hop"", … "".""]","[3000, 3000, … 26]",1
35766,"""think about it.. he´s called J…","[""think"", "".."", … ""elect""]","[2688, 27, … 911]",0
23678,"""Hmm, you could rewrite this wi…","[""hmm"", "","", … ""justsaying""]","[1284, 22, … 3000]",0
52396,"""Every girl in a game, or on si…","[""girl"", ""game"", … "".""]","[1164, 1135, … 26]",0
…,…,…,…,…
6785,"""I just think my dick couldn't …","[""just"", ""think"", … ""..😂😂😂""]","[1482, 2688, … 3000]",0
31355,"""no he deadasss hit the nail on…","[""deadass"", ""hit"", … "".""]","[3000, 1282, … 26]",0
49834,"""Aren't these goblins always wh…","[""goblins"", ""whine"", … ""hastagsarestillcool""]","[3000, 2914, … 3000]",0


In [5]:
tfidf_pipeline().fit(token_lists(tra))

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('count',
                 CountVectorizer(lowercase=False, max_features=3000,
                                 stop_words='english',
                                 tokenizer=<label_legends.preprocess.ConlluTokenizer object at 0x7b9fe8e6aba0>,
                                 vocabulary={'!': np.int64(0),
                                             '!!': np.int64(1),
                                             '!!!': np.int64(2),
                                             '!!!!': np.int64(3),
                                             '!!!!!': np.int64(4),
                                             '!?': np.int64(5),
                                             '"': np.int64(6), '#': np.int64(7),
                                             '#trump': np.int64(8),
                                             '#x200b...
                                             '&': np.int64(12),
                                             "'": np.int64(13),
                                             "''": np.int64(14),
                                             "'s": np.int64(15),
                                             '(': np.int64(16),
                                             ')': np.int64(17),
                                             '*': np.int64(18),
                                             '**': np.int64(19),
                                             '**.': np.int64(20),
                                             '+': np.int64(21),
                                             ',': np.int64(22),
                                             '-': np.int64(23),
                                             '--': np.int64(24),
                                             '---': np.int64(25),
                                             '.': np.int64(26),
                                             '..': np.int64(27),
                                             '...': np.int64(28),
                                             '....': np.int64(29), ...})),
                ('tfidf', TfidfTransformer())])

In [6]:
X_train = tfidf_pipeline().transform(token_lists(tra))
X_test = tfidf_pipeline().transform(token_lists(test))

In [7]:
clf = XGBClassifier()
clf.fit(X_train, tra.select("label").to_series().to_torch())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
y_pred = clf.predict(X_test)
y_true = test["label"].to_list()

In [9]:
scores = calculate_scores(y_true, y_pred)
print(scores)

precision:	0.7639
recall:		0.3884
fscore:		0.5150
accuracy:	0.8101
tn: 8511	 fp: 374
fn: 1905	 tp: 1210


In [153]:
with mlflow.start_run(
        experiment_id=get_experiment().experiment_id, 
        run_name="xgboost testing"
        ):
    mlflow.log_params(clf.get_xgb_params())
    mlflow.log_metrics(scores.asdict())
    mlflow.set_tag("model", "XGBoost")
    signature = infer_signature(X_train, clf.predict(X_train))
    mlflow.log_table({"id": test["id"], "prediction": y_pred}, "predictions.json")

    # log the model
    model_info = mlflow.xgboost.log_model(
        clf,
        "",
        signature=signature,
        registered_model_name="xgboost",
    )

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:01:18] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


Registered model 'xgboost' already exists. Creating a new version of this model...
Created version '10' of model 'xgboost'.


In [158]:
# Load table directly from registry (optional)
get_latest_run("xgboost")

'cc114229371740a78f2029e074a0bcb5'

In [161]:
# Download the most recent predictions of the model
download_predictions("xgboost")
load_predictions("xgboost")

id,prediction
i64,i64
100,0
10005,1
10006,1
10007,1
10008,0
…,…
9985,0
9986,0
9993,0
